# Библиотеки

In [1]:
import pickle
from tqdm.auto import tqdm

import numpy as np
import pandas as pd
import polars as pl

from lemma import LemmaRecommender

/home/jupyter/.local/lib/python3.10/site-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


# Обучение

In [2]:
features = pl.read_parquet('./features.parquet', columns = ['video_id', 'v_pub_datetime']).sort('v_pub_datetime')
features.tail()

video_id,v_pub_datetime
str,str
"""video_2172262""","""2023-11-22 15:…"
"""video_22618974…","""2023-11-22 15:…"
"""video_11792065…","""2023-11-22 15:…"
"""video_11792064…","""2023-11-22 15:…"
"""video_11792080…","""2023-11-22 15:…"


In [3]:
features = features.unique()

In [4]:
videos = pl.read_parquet('./videos.parquet', columns = ['video_id', 'video_title', 'v_pub_datetime']).sort('v_pub_datetime')
videos.tail(1_000_000)

video_id,video_title,v_pub_datetime
str,str,"datetime[ns, +00:00]"
"""video_27609121…","""LIVE 3 : Gayat…",2023-11-12 22:53:48 +00:00
"""video_16206525…","""6 - Шримати Ра…",2023-11-12 22:53:49 +00:00
"""video_15284998…","""7 CHAKRAS HEAL…",2023-11-12 22:53:49 +00:00
"""video_13865933…","""хадсон — Море…",2023-11-12 22:53:49 +00:00
"""video_18611538…","""KALIYUGA || MO…",2023-11-12 22:53:49 +00:00
"""video_5992598""","""Dmitri Hvorost…",2023-11-12 22:53:49 +00:00
"""video_21456410…","""नवरात्रि स्पॆश…",2023-11-12 22:53:49 +00:00
"""video_32520621…","""38 лет в браке…",2023-11-12 22:53:49 +00:00
"""video_10561672…","""Большое спасиб…",2023-11-12 22:53:49 +00:00


за последние два дня целых миллион видео???

In [5]:
videos = videos.join(features, on='video_id', how='left')
videos.tail()

video_id,video_title,v_pub_datetime,v_pub_datetime_right
str,str,"datetime[ns, +00:00]",str
"""video_14412320…","""Угадай мелодию…",2023-11-14 11:58:56 +00:00,null
"""video_15300130…","""Гога, Киря и Е…",2023-11-14 11:59:13 +00:00,null
"""video_15600786…","""Специальный ре…",2023-11-14 11:59:26 +00:00,null
"""video_15299879…","""Угадай мелодию…",2023-11-14 11:59:41 +00:00,null
"""video_11792077…","""Давай поженимс…",2023-11-14 11:59:59 +00:00,null


In [6]:
videos = videos.filter((~pl.col('v_pub_datetime').is_null()) & (~pl.col('v_pub_datetime_right').is_null()))
videos.tail(1_000_000)

video_id,video_title,v_pub_datetime,v_pub_datetime_right
str,str,"datetime[ns, +00:00]",str
"""video_27711768…","""Авто из Кореи …",2023-09-27 10:30:42 +00:00,"""2023-09-27 10:…"
"""video_16046805…","""Русский язык_5…",2023-09-27 10:30:43 +00:00,"""2023-09-27 10:…"
"""video_32024614…","""Apera Instrume…",2023-09-27 10:30:43 +00:00,"""2023-09-27 10:…"
"""video_28578316…","""СМОЖЕШЬ СДЕЛАТ…",2023-09-27 10:30:44 +00:00,"""2023-09-27 10:…"
"""video_370606""","""Жди Меня 05.01…",2023-09-27 10:30:44 +00:00,"""2023-09-27 10:…"
"""video_15732705…","""Сергей Банарь …",2023-09-27 10:30:44 +00:00,"""2023-09-27 10:…"
"""video_24676052…","""Вопрос №2 § 27…",2023-09-27 10:30:45 +00:00,"""2023-09-27 10:…"
"""video_5606659""","""Как Сделать Ро…",2023-09-27 10:30:46 +00:00,"""2023-09-27 10:…"
"""video_7076222""","""ПАРА ТУФЕЛЬ. Д…",2023-09-27 10:30:47 +00:00,"""2023-09-27 10:…"


теперь выглядит более реалистично

In [7]:
video_ids = videos.tail(1_000_000)["video_id"].to_list()
video_titles = videos.tail(1_000_000)["video_title"].to_list()

In [8]:
lemma_rec = LemmaRecommender()

In [9]:
lemma_rec.fit(video_ids, video_titles)

  0%|          | 0/1000000 [00:00<?, ?it/s]

# Тестирование

In [10]:
videos = videos.tail(1_000_000).to_pandas()
videos = videos.set_index('video_id')

In [11]:
%%time
recs = lemma_rec.predict(['цифровой прорыв', 'искусственный интеллект', ''], 10)
recs

  0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 133 ms, sys: 12.2 ms, total: 145 ms
Wall time: 143 ms


query_id,video_id,lex_score,lex_rank
i32,str,f32,i8
0,"""video_20498139…",16.428774,1
0,"""video_16193787…",12.2701,2
0,"""video_6941488""",10.966259,3
0,"""video_15480127…",10.966259,4
0,"""video_22062260…",10.966259,5
0,"""video_31103050…",10.966259,6
0,"""video_27330270…",10.966259,7
0,"""video_12838041…",10.966259,8
0,"""video_31827393…",10.966259,9


In [12]:
recs = recs.to_pandas().groupby('query_id')[['video_id', 'lex_score']].agg(list)

In [13]:
ids, scores = recs['video_id'], recs['lex_score']

In [14]:
ids, scores

(query_id
 0    [video_20498139, video_16193787, video_6941488...
 1    [video_12219378, video_22102798, video_1120109...
 Name: video_id, dtype: object,
 query_id
 0    [16.428773880004883, 12.270099639892578, 10.96...
 1    [24.1533260345459, 22.119359970092773, 20.4013...
 Name: lex_score, dtype: object)

In [15]:
videos.loc[ids[0], 'video_title'], videos.loc[ids[0], 'video_title']

(video_id
 video_20498139    Стартовал новый сезона проекта "Цифровой проры...
 video_16193787                   Genshin Impact: Прорыв вдохновения
 video_6941488                                        Цифровой рубль
 video_15480127                                    Цифровой манометр
 video_22062260                                     Цифровой паспорт
 video_31103050                                    Цифровой паспорт?
 video_27330270                                   Цифровой акробат))
 video_12838041                                       Цифровой полис
 video_31827393                                 Цифровое расширение!
 video_14578919                                Цифровой Баян-оркестр
 Name: video_title, dtype: object,
 video_id
 video_20498139    Стартовал новый сезона проекта "Цифровой проры...
 video_16193787                   Genshin Impact: Прорыв вдохновения
 video_6941488                                        Цифровой рубль
 video_15480127                                 

In [16]:
scores[0], scores[1]

([16.428773880004883,
  12.270099639892578,
  10.966259002685547,
  10.966259002685547,
  10.966259002685547,
  10.966259002685547,
  10.966259002685547,
  10.966259002685547,
  10.966259002685547,
  10.966259002685547],
 [24.1533260345459,
  22.119359970092773,
  20.401351928710938,
  20.401351928710938,
  20.401351928710938,
  18.93098258972168,
  18.93098258972168,
  18.93098258972168,
  18.93098258972168,
  18.93098258972168])

In [17]:
queries = pd.read_csv('./submission_queries.csv')
queries

,query
0,экстрасенсы битва сильнейших 2 сезон
1,битва экстрасенсов 2
2,Экстрасенсы
3,битва сильнейших 19 сезон
4,битва экстрасенсов 20 сезон
...,...
1995,ПОЛИКЛИНИКА в этом ведре и есть счастье ?
1996,Как избавиться от страхов и начать бизнес ?
1997,Как и где продавать аромасвечи ?
1998,Ответы на Ваши комментарии! Аромасвечи


In [18]:
%%time
recs = lemma_rec.predict(queries['query'], 10)
recs

  0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 4.64 s, sys: 378 ms, total: 5.02 s
Wall time: 4.84 s


query_id,video_id,lex_score,lex_rank
i32,str,f32,i8
0,"""video_13778122…",28.924025,1
0,"""video_16372109…",26.677498,2
0,"""video_5067234""",26.677498,3
0,"""video_3599632""",26.677498,4
0,"""video_27119025…",26.677498,5
0,"""video_22989064…",26.677498,6
0,"""video_26919518…",24.754795,7
0,"""video_23769754…",24.754795,8
0,"""video_18764678…",24.754795,9


In [19]:
recs = recs.to_pandas().groupby('query_id')[['video_id', 'lex_score']].agg(list)

In [21]:
i = np.random.randint(0, len(queries))
queries.loc[i, 'query'], videos.loc[recs.loc[i, 'video_id'], 'video_title']

('импровизация 4 сезон 33 серия',
 video_id
 video_5698521       Воскресший Эртугрул, 4 сезон, 33 серия
 video_29427975               Анжелика | Сезон 2 | Серия 33
 video_12899074    САШАТАНЯ, 5 сезон, 33 серия (18.12.2019)
 video_13782407      Воскресший Эртугрул, 5 сезон, 33 серия
 video_28093705      Воскресший Эртугрул, 3 сезон, 33 серия
 video_4180841       Воскресший Эртугрул, 1 сезон, 33 серия
 video_8706694       Воскресший Эртугрул, 2 сезон, 33 серия
 video_28189842      Наш спецназ 3 сезон 1 серия (33 серия)
 video_14321547                    Убийство 4 сезон 4 серия
 video_10250514                     Клиника 4 сезон 4 серия
 Name: video_title, dtype: object)

# Сохранение

In [22]:
with open('lemma_rec_1e6.pickle', 'wb') as f:
    pickle.dump(lemma_rec, f)